In [26]:
!pip install opencv-python
#!pip install opencv-python==3.4.2.16
#!pip install opencv-contrib-python==3.4.2.16
import cv2
import numpy as np
import pygco
import math

In [27]:
def get_edges(image):
    vh = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=5)#hyperparameter 5
    vv = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=5)#hyperparameter 5
    color_variations = np.zeros(np.shape(vh))
    for row in range(np.shape(vh)[0]):
    	for col in range(np.shape(vh)[1]):
    		color_variations[row,col] = math.sqrt(vh[row,col]**2 + vv[row,col]**2)
            #color_variations[row,col] = vh[row,col]*0.5 + vv[row,col]*0.5
    return color_variations

In [28]:
def get_edges2(image, blur_width=3):
        img_blurred = cv2.GaussianBlur(image, (0, 0), blur_width)
        vh = cv2.Sobel(img_blurred, -1, 1, 0)
        vv = cv2.Sobel(img_blurred, -1, 0, 1)

        #vh = vh/np.max(vh)
        #vv = vv/np.max(vv)

        v = 0.5*vv + 0.5*vh
        
        return v

In [29]:
def get_pairwise_costs(centroids, centroid_num, l = 1):
	pairwise_costs = np.zeros((centroid_num, centroid_num))
	for i in range(centroid_num):
		for j in range(centroid_num):
			c1 = np.array(centroids[i])
			c2 = np.array(centroids[j])
			pairwise_costs[i,j] = np.linalg.norm(c1-c2)
	return (l*pairwise_costs).astype('int32')

In [2]:
def get_index(r, c, col_no):
    return r*col_no + c

def get_weight(r1,c1,r2,c2,color_vars):
    if  color_vars[r1, c1] == 0 or color_vars[r2, c2] == 0:
        return int(100000000000)
    return int((color_vars[r1, c1] + color_vars[r2, c2])/2)
    return int((1. / color_vars[r1, c1] + 1. / color_vars[r2, c2]) / 2.)

SyntaxError: invalid syntax (<ipython-input-2-2b1c5709d0b5>, line 5)

In [31]:
def graph_cut(image, unary_costs, centroids, centroid_num, alpha = 128):
	
    #get color variations
    color_vars = get_edges(image)
    
    #unary costs is a 3d matrix
    #make it 2d, each row is unary cost of a pixel now.
    row_no = np.shape(unary_costs)[0]
    col_no = np.shape(unary_costs)[1]
    unary_cost_list = np.zeros((row_no*col_no, np.shape(unary_costs)[2]))
    

    edges =[]
    for r in range(row_no):
        for c in range(col_no):
            unary_cost_list[get_index(r, c, col_no), :] = unary_costs[r, c, :]

            row_safe = r != row_no - 1
            col_safe = c != col_no - 1
            if (row_safe):
                edges.append([get_index(r,c,col_no), get_index(r+1,c,col_no), get_weight(r,c,r+1,c,color_vars)])
            if (col_safe):
                edges.append([get_index(r,c,col_no), get_index(r,c+1,col_no), get_weight(r,c,r,c+1,color_vars)])
            if (row_safe and col_safe):
                edges.append([get_index(r,c+1,col_no), get_index(r+1,c,col_no), get_weight(r,c+1,r+1,c,color_vars)])
                edges.append([get_index(r+1,c,col_no), get_index(r,c+1,col_no), get_weight(r+1,c,r,c+1,color_vars)])

    edges_int32 = np.array(edges).astype('int32')
    pairwise_costs_int32 = get_pairwise_costs(centroids, centroid_num)
    unary_cost_list_int32 = (alpha*unary_cost_list).astype('int32')
    
    #perform graphcut optimization
    labels_list = pygco.cut_from_graph(edges_int32, unary_cost_list_int32, pairwise_costs_int32, n_iter=-1, algorithm='swap')

    labels = np.zeros((row_no, col_no))
    for r in range(row_no):
        for c in range(col_no):
            labels[r,c] = labels_list[get_index(r,c,col_no)]

    return labels.astype('int')

In [32]:
def graph_cut2(image, unary_costs, centroids, centroid_num,alpha=128):
    
    #get color variations
    color_vars = get_edges2(image)
    
    #calculate pairwise potiential costs (distance between color classes)
    pairwise_costs_int32 = get_pairwise_costs(centroids, centroid_num)
    
    unary_costs_int32 = (alpha*unary_costs).astype('int32')#alpha 128
    
    edgesY_int32 = color_vars.astype('int32')
    edgesX_int32 = color_vars.astype('int32')
    
    #perform graphcut optimization
    labels = pygco.cut_simple_vh(unary_costs_int32, pairwise_costs_int32, edgesY_int32, edgesX_int32, n_iter=10, algorithm='swap') 

    return labels